# Batch SQL Gold


In [0]:

USE IDENTIFIER(:catalog ||'.'|| :schema)


# Customers

In [0]:
MERGE INTO gold_customers AS T
USING (
  WITH changed_register_for_merge AS (
    SELECT 
      sha2(concat_ws('|',Customer_ID,Registration_TimeStamp),256) as Customer_SK,
      Customer_ID,
      Social_Number,
      Name,
      email,
      Address,
      Moviment_Date,
      Registration_TimeStamp AS StartDate,
      Null AS EndDate,
      sha2(concat_ws('|',*),256) as hash,  --> HASH HERE DEPENDS ONLY ON THE COLUMNS OF SILVER TABLE
      Operation
    FROM silver_customers
    WHERE Moviment_Date = :moviment_date

    UNION ALL

    SELECT DISTINCT T.*, Null AS Operation
    FROM ( SELECT Customer_ID, Registration_TimeStamp FROM silver_customers WHERE Moviment_Date = :moviment_date ) S
    LEFT JOIN gold_customers T 
      ON S.Customer_ID = T.Customer_ID
    WHERE S.Registration_TimeStamp > T.StartDate AND S.Registration_TimeStamp < T.EndDate
  ),
  lead_lag_table AS (
  SELECT *,
    LEAD(StartDate) OVER (PARTITION BY Customer_ID ORDER BY StartDate) AS EndDate1,
    LAG(EndDate) OVER (PARTITION BY Customer_ID ORDER BY StartDate) AS EndDate2
  FROM changed_register_for_merge
  )
  SELECT
    Customer_SK,
    Customer_ID,
    Social_Number,
    Name,
    email,
    Address,
    Moviment_Date,
    StartDate,
    COALESCE(EndDate1, EndDate2, '9999-12-31') AS EndDate,
    hash
  FROM lead_lag_table
  WHERE Operation <> 'DELETE'
) AS S
ON T.Customer_ID = S.Customer_ID AND S.StartDate = T.StartDate

WHEN MATCHED THEN UPDATE SET T.EndDate = S.EndDate
-- Inserting:
WHEN NOT MATCHED THEN INSERT *

In [0]:
UPDATE IDENTIFIER(:catalog ||'.control.control_tables_moviments')
SET MOVIMENT_DATE = :moviment_date
WHERE Project_Name = :project
  AND Table_name = 'gold_customers'